# 3-2 厳選! 頻出テクニック(1)

### しゃくとり法

<img src="images\3-2\2023-05-03-16-29-47.png" width="800">

### 解説

要素がすべて正の数なので、ある部分列(s, t) = $a_s...a_t\geqq S$であるならば、$t<t'$に対しても$a_s...a_{t'}\geqq S$

In [5]:
# sum[t]を探すときに二分探索を用いる方法
from bisect import bisect_right


def solve(n, S, a):
    # a_0 ~ a_iのsumをあらかじめ計算しておくと計算量が節約できる
    sum_result = [0]
    for i in range(n):
        sum_result.append(sum_result[i] + a[i])

    if sum_result[n] < S:
        # 解が存在しない
        print(0)
        return

    res = n
    for s, sum_s in enumerate(sum_result):
        if S + sum_s > sum_result[-1]:
            break
        t = bisect_right(sum_result, S + sum_s)
        res = min(res, t - s)
    print(res)


# 入力

 N = 10
S = 15
a = [5, 1, 3, 5, 10, 7, 4, 9, 2, 8]
"""
n = 5
S = 11
a = [1, 2, 3, 4, 5]
"""

solve( N, S, a)

2


In [17]:
# しゃくとり法
def solve(n, S, a):
    res = n + 1
    s = 0
    sum_result = 0
    for t in range(n):
        # 配列を順に足していく
        sum_result += a[t]
        while sum_result >= S:
            # 合計がSを超えていたら resを更新
            res = min(res, t - s + 1)
            # 部分列の幅を合計がS未満になるまで後ろから狭めていく
            sum_result -= a[s]
            s += 1

    # resが初期値から変化なければ、解なしとして0を表示
    if res > n:
        res = 0
    print(res)


# 入力
 N = 10
S = 15
a = [5, 1, 3, 5, 10, 7, 4, 9, 2, 8]
"""
n = 5
S = 11
a = [1, 2, 3, 4, 5]
"""


solve( N, S, a)

2


### Jessica's Reading Problem (POJ No. 3320)
<img src="images\3-2\2023-05-03-21-45-00.png" width="800">

### 解説

ページを読む範囲start-endでの項目の出現回数を数える。全項目が1回以上出現している場合は範囲を調整するということを行えば二分探索で解くことができる。

出現回数のカウントは辞書で行う。pythonではdefaultdict(int)で辞書の初期値を0と設定するとやりやすい。

In [4]:
from collections import defaultdict


def solve(P, a):
    n = len(set(a))  # 書かれている項目の総種類数

    start = 0  # 読むのを始めるページ
    end = 0  # 読むのを終わるページ
    num = 0  # start-end間に含まれる項目数

    count = defaultdict(int)  # 項目の出現回数のカウント用。defaultdict(int)でキーが増えた時の初期値を0に設定している
    res = len(a) + 1
    for end in range(P):
        count[a[end]] += 1
        if count[a[end]] == 1:
            num += 1

        # start-endで全項目カバーできている間はループする
        while num == n:
            # 現時点での解(ページ数)を更新
            res = min(res, end - start + 1)
            # start位置を次に進めて範囲を絞っていく
            count[a[start]] -= 1  # カウントを減らす
            if count[a[start]] == 0:
                num -= 1
            start += 1

    if res > n:
        res = 0
    print(res)
    return


# 入力
P = 5
a = [1, 8, 8, 8, 1]

solve(P, a)

2


### 反転

<img src="images\3-2\2023-05-05-16-00-01.png" width="800">


### 解説

- 反転する順序は順不同でよい
- 同じ区間(Kの幅)は0~1回しか反転しなくていい
- 一番左の牛はどんなKでも1区間にしか含まれない。つまり一番左の区間の操作(1手目)は確定する

このことから、次の手順で解くことができる

1. Kを固定する
2. 牛0から最後の牛N - K + 1(最後の牛がギリギリ含まれる区間の一番左の牛)まで確認していく
3. 牛iが後ろを向いていれば区間[i, i + k - 1]の牛を反転 → 操作回数を+1する
4. 最後(牛N - K + 1)まで操作が終わったあと、すべての牛が前を向いていれば操作回数を更新する
   
ただしこの手順の場合、計算量はKを調べるK=1~N, 牛iを調べる(i=0~N-K+1), 1区間の反転操作の回数が最大K。これを繰り返すことになるので、

$$
\sum_{K=1}^N (N-K+1)・K=O(N^3)
$$
となってしまう。この問題ではN=最大5000なので時間内に終わらない

計算量を落とすためには、区間Kの牛を反転させる"$\sum_{K=1}^N (N-K+1)・「K」=O(N^3)$"の、左辺一番右のKを1単位に読み替えることにする。つまり、牛iから牛N-K+1の区間の反転回数f(i)について、

$$f[i]:=区間[i, i+K-1]を反転させたなら1, そうでないなら0$$

と定義する。

手順1~3で牛iの向きを確認したとき、牛iが最初の向きと逆になっているのは、$\sum_{j=max(0, i-K+1)}^{i-1}f[j]$(注目が牛iに到達するまでに、牛iを含む範囲Kでの反転有無fの合計)が奇数の場合で、それ以外は最初の向きのままであることに注目する。

$$
g[i] = \sum_{j=max(0, i-K+1)}^{i-1}f[j]の遇奇を判定すればよい
$$

さて、ここでg[i]が計算済みのとき、g[i+1]は以下で求めることができる。

$$
g[i + 1] = 
\left 
    \{
    \begin{array}{ll}
        g[i] + f[i] - f[i - K + 1] & (i-K+1 \geq 0) \\
        g[i] + f[i] & (i - K + 1 < 0)
    \end{array}
\right.
$$

よって牛iの反転操作(g[i]の計算)は$O(1)$となり、全体の計算量は、
$$
\sum_{K=1}^N (N-K+1)=O(N^2)
$$
となる


In [3]:
def calc(N, K, direction):
    f = [0] * N  # 区間[i,i+K-1]を反転させたかどうか
    res = 0
    g = 0  # fの和g
    for i in range(N - K + 1):  # 区間[i, i+K-1]に着目
        f[i] = (direction[i] + g) % 2
        # 1=先頭の牛が後ろを向いている
        res += f[i]

        # gの計算
        g += f[i]
        if i - K + 1 >= 0:
            g -= f[i - K + 1]

    # 残りの牛が前を向いているかをチェック
    for i in range(N - K + 1, N):
        if (direction[i] + g) % 2 != 0:
            # 1頭でも後ろ向き=1であれば解なし
            return -1
        if i - K + 1 >= 0:
            g -= f[i - K + 1]
    return res


def solve(N, A):
    ans_k, ans_m = 1, N
    cow_directions = [c == "B" for c in A]  # 牛の向きをboolに変換
    # すべてのKを調査
    for K in range(1, N + 1):
        M = calc(N, K, cow_directions)

        # Mがより小さければ答えを更新
        if M >= 0 and ans_m > M:
            ans_k, ans_m = K, M

    print(ans_k, ans_m)


# 入力
N = 7
A = "BBFBFBB"

solve(N, A)

3 3


<img src="images/3-2/2023-05-20-08-58-37.png" width="800">

### 解説

任意のマスを反転するとき、上下左右にマスがあれば一緒に反転するという操作がキモ。

Face The Right Way同様、同じマスを複数回反転させる必要はない。反転の順序が結果に影響しないことも同じなので、全部で$2^{NM}$通りのひっくり返し方があるが、これでは計算量が大きいので削減していく。

Face The Right Wayと異なるのは、左上端(1, 1)のマスのひっくり返し方が1通りではないこと。つまり、(1, 2)や(2, 1)をひっくり返しても良い。

さて、反転順序は自由で良いので、左上から1行ずつ試行していくとする。

ここで、一番上の1行目のマスの反転の仕方は$2^N$通りあるが、ある1通りについて考える。次の手順として2行目のマスを反転するかどうか考えると、そのマスの真上の1行目マスが黒である時には反転しなければいけないことがわかる。以下3行目以降も同様。これを最終行まで行った結果、最終行に黒のマスが残っていればその手順は解ではないとわかる。

以上から、1行目のマスの反転の仕方さえ決まれば、他の行の手順も決まっていくことがわかる。計算量は1行目のマスの反転の仕方全$2^N$通りにマスの数を掛けて、$O(MN2^N)$となる

1. 1行目のマスを反転させか方を決める$2^N$通り
2. 2~最終行までの反転手順を決める(1つ上の行が全て白になるように)
3. 最終行が全て白になったか確認
4. 3が正、かつ反転回数が少なければ解を更新

In [13]:
from copy import deepcopy

def getAboveTileStatus(x, y, TILE, flip):
    """
    x, yのマスの色を影響範囲の反転有無から計算
    """
    # あるマス(x, y)の色に影響を及ぼすマスを全てリストアップ
    # (左, 下, 中央, 上, 右)
    RANGE_LIST = ((-1, 0), (0, -1), (0, 0), (0, 1), (1, 0))
    flip_cnt = TILE[x][y]  # 初期値
    # 四方+中央のマスの座標を順に計算
    for dx, dy in  RANGE_LIST:
        x2, y2 = x + dx, y + dy
        # 四方＋中央のマス(x2, y2)がタイルの範囲内か判定
        if 0 <= x2 < M and 0 <= y2 < N:
            # 四方+中央のマスの反転有無(0/1)を加算
            flip_cnt += flip[x2][y2]
    return flip_cnt % 2  # 対象マス(x, y)の色は(初期状態 + 反転回数合計)の偶奇で決まる


def calc(M, N, TILE, flip):
    """
    1行目を決めたときの最小手数を計算
    (解なしなら-1を返す)
    """
    # 2行目からM行目まで確認
    for row in range(1, M):
        # 1列目からN列まで確認
        for col in range(N):
            # 直上のマスが1なら(row, col)起点に反転操作
            if getAboveTileStatus(row - 1, col, TILE, flip):
                flip[row][col] = 1
    
    # 全ての反転操作終了後，最終行が全て0かチェック
    for col in range(N):
        if getAboveTileStatus(M - 1, col, TILE, flip):
            # 1があれば解なし
            return -1
    # 反転回数を出力
    res = sum(sum(flip_row) for flip_row in flip)
    return res


# --- 処理 ---
## 解を求める
def solve(M, N, TILE):
    opt = [[""] * N for _ in range(M)]  # 最適解
    res = -1  # 初回判定のための初期値

    # 反転したかの記録用変数を初期化(0を N x (M - 1) 配置)
    # 1行目は後から追加するのでN-1行分用意
    flip = [[0] * N for _ in range(M - 1)]

    # 1行目マスの反転の仕方を辞書順で全(2^N)通り試す
    # 1 << N, N=3の時 1000=10進数で8=2^3
    # 2 ** Nで計算するより速い
    for i in range(1 << N):
        # 何やっているか分かりづらいが、iをN桁の2進数にして配列に直している
        first_row = [int(x) for x in f"{i:0{N}b}"]
        flip = [first_row] + flip 

        # 反転の操作箇所の決定と反転回数の計算
        num = calc(M, N, TILE, flip)
        if num >= 0 and (res < 0 or res > num):
            # 解あり & (初回 or もっと短い手数) の場合は更新
            res = num
            opt = deepcopy(flip)

    ## 出力
    if res < 0:
        print("IMPOSSIBLE")
    else:
        for l in opt:
            print(*l)


# 入力
M, N = 4, 4
TILE = [
    [1, 0, 0, 1],
    [0, 1, 1, 0],
    [0, 1, 1, 0],
    [1, 0, 0, 1],
]

solve(M, N, TILE)

0 0 0 0
1 0 0 1
1 0 0 1
0 0 0 0
